## 자료 구조 확인

In [1]:
import pandas as pd

# CSV 파일 로드
file_path = "2010to2025_results.csv"
df = pd.read_csv(file_path)

# 컬럼 이름 출력
print(df.columns)


Index(['title', 'contents', 'date'], dtype='object')


In [ ]:
# import pandas as pd

# # CSV 파일 로드
# file_path = "processed_data.csv"
# df = pd.read_csv(file_path)

# # 컬럼 이름 출력
# print(df.columns)


Index(['contents', 'tokens'], dtype='object')


## 기본

In [6]:
import pandas as pd
from ekonlpy import Mecab
import re
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime

# 형태소 분석기 초기화
mecab = Mecab()

# 파일 경로
file_path = r"C:\Users\yoons\Desktop\금리예측프로젝트\filter\2010to2025_results.csv"
output_path = r"C:\Users\yoons\Desktop\금리예측프로젝트\filter\really_data_test.csv"

# 텍스트 전처리 함수 (대괄호 및 소괄호 내 내용 삭제)
def cleaned_(text: str):
    pattern1 = r'\[[^\]]*\]' 
    pattern2 = r'\([^)]*\)' 
    text = re.sub(pattern1, '', text)
    text = re.sub(pattern2, '', text)
    return text.strip()

# 품사 태깅 및 불필요한 품사 필터링
def process_content(content: str):
    content = cleaned_(content)  # 텍스트 전처리
    lemmatized = mecab.lemmatize(content)  # 품사 태깅 및 표제어 추출
    
    # 필요한 품사만 필터링: 명사, 동사, 형용사만
    filtered_tags = [tag for tag in lemmatized if tag[1][0] not in ['I', 'J', 'E', 'S']]
    return filtered_tags

# 파일 읽기 및 데이터 처리
start = datetime.now()

# 데이터 로드
korea = pd.read_csv(file_path)

# 병렬 처리로 텍스트 처리
with ProcessPoolExecutor() as executor:
    contents_tagged = list(executor.map(process_content, korea['contents']))

# 처리된 결과를 DataFrame에 추가
korea['tagged'] = contents_tagged

# 결과 파일로 저장
korea.to_csv(output_path, encoding='utf-8-sig', index=False)

end = datetime.now()
print(f"소요 시간: {end - start}")


BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [ ]:
import pandas as pd
from ekonlpy import Mecab  # Mecab으로 대체
import re

# 파일 경로
file_path = r"C:\Users\yoons\Desktop\금리예측프로젝트\filter\2010to2025_results.csv"
output_path = r"C:\Users\yoons\Desktop\금리예측프로젝트\filter\real_data_test.csv"

# 형태소 분석기 초기화
mecab = Mecab()

# 청크 크기 설정
chunk_size = 10000

# 첫 번째 청크의 헤더를 출력할 수 있도록 처리
for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    chunk.dropna(subset=['title', 'contents', 'date'], inplace=True)
    chunk['contents'] = chunk['contents'].str.replace(r'[^ㄱ-ㅎ가-힣0-9A-Za-z ]', '', regex=True)
    chunk['tagged'] = chunk['contents'].apply(lambda x: mecab.pos(x))  # pos()로 품사 태깅

    # 첫 번째 청크에 헤더 저장
    if i == 0:
        chunk.to_csv(output_path, encoding='utf-8-sig', index=False)
    else:
        # 이후 청크는 헤더 없이 추가 저장
        chunk.to_csv(output_path, encoding='utf-8-sig', index=False, mode='a', header=False)
    
    print(f"청크 {i+1} 처리 완료. 저장된 행 수: {len(chunk)}")

print(f"전체 데이터 처리 완료! 결과가 {output_path}에 저장되었습니다.")


청크 1 처리 완료. 저장된 행 수: 9992
청크 2 처리 완료. 저장된 행 수: 9926
청크 3 처리 완료. 저장된 행 수: 9961
청크 4 처리 완료. 저장된 행 수: 10000
청크 5 처리 완료. 저장된 행 수: 10000
청크 6 처리 완료. 저장된 행 수: 10000
청크 7 처리 완료. 저장된 행 수: 10000
청크 8 처리 완료. 저장된 행 수: 4306
전체 데이터 처리 완료! 결과가 C:\Users\yoons\Desktop\금리예측프로젝트\filter\real_data_test.csv에 저장되었습니다.


In [19]:
import pandas as pd
import ast
from collections import Counter

# 저장된 CSV 파일 불러오기
output_path = r"C:\Users\yoons\Desktop\금리예측프로젝트\filter\real_data_test.csv"
final_df = pd.read_csv(output_path)

# 'tagged' 컬럼을 리스트로 변환 (CSV 저장 시 문자열로 변환되었을 가능성 있음)
final_df['tagged'] = final_df['tagged'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 'tokens' 컬럼 생성 (형태소 분석 결과에서 단어만 추출)
final_df['tokens'] = final_df['tagged'].apply(lambda x: [word for word, pos in x])

# 제거할 단어 리스트
remove_words = {"'", "또", "크", "N", "기자", "뉴스", "/", ",", " ", "G", "V"}

# 전체 데이터에서 토큰화된 단어 리스트를 모은 후 빈도 계산
all_tokens = [token for sublist in final_df['tokens'] for token in sublist]
word_counts = Counter(all_tokens)

# 특정 단어 제거
for word in remove_words:
    if word in word_counts:
        del word_counts[word]

# 상위 100개 단어 출력
print(word_counts.most_common(100))


[('이', 1281696), ('다', 1175029), ('는', 1022096), ('을', 869568), ('에', 741081), ('은', 693950), ('의', 650640), ('하', 560586), ('를', 549052), ('가', 537845), ('있', 452139), ('했', 445265), ('고', 421689), ('으로', 413161), ('한', 322602), ('것', 302292), ('로', 300751), ('도', 290190), ('에서', 260524), ('들', 201555), ('등', 188464), ('원', 187755), ('과', 183592), ('금리', 173755), ('년', 166135), ('할', 165616), ('월', 164073), ('일', 162055), ('만', 157587), ('인', 152409), ('수', 144019), ('적', 142781), ('기', 137835), ('억', 137241), ('와', 130885), ('다고', 130626), ('되', 128805), ('게', 124595), ('지', 123309), ('1', 118683), ('면', 118308), ('미국', 117203), ('해', 109901), ('습니다', 100797), ('보다', 99050), ('어', 95496), ('라고', 94879), ('다는', 92345), ('3', 91751), ('말', 90752), ('2', 87874), ('까지', 87275), ('상승', 85004), ('면서', 84855), ('대출', 83759), ('시장', 81805), ('경제', 81458), ('않', 79425), ('었', 78752), ('지만', 76245), ('높', 71801), ('전망', 64128), ('달러', 63780), ('인상', 63139), ('투자', 63122), ('하락', 61997), ('됐', 

## 불용어 제거

In [ ]:
# import ast

# # 문자열을 리스트로 변환
# final_df['tokens'] = final_df['tokens'].apply(ast.literal_eval)

# # 변환 확인
# print(type(final_df['tokens'][0]))  # 이제 리스트여야 함


<class 'list'>


In [ ]:
print(final_df['tagged'].head())  # 상위 5개 데이터 출력
print(type(final_df['tagged'][0]))  # 첫 번째 값의 데이터 타입 확인


NameError: name 'chunk' is not defined

In [2]:
print(df['tokens'].head(10))


NameError: name 'df' is not defined

In [16]:
import pandas as pd
from collections import Counter

# 제외할 품사 목록
exclude_pos = {"XSV", "VCN", "VCP"}

# 제외할 특정 단어 목록
exclude_words = {"다", "또", "크","또"}  # 여기에 제외하고 싶은 단어들을 추가하세요.

# CSV 파일 불러오기
df = pd.read_csv("processed_data.csv")

# 필터링하여 새로운 리스트 생성
filtered_tokens = []
for sublist in df['tokens']:
    try:
        tokens = eval(sublist)  # 문자열을 리스트로 변환 (예: "['단어/NNG', '형용사/VA']")
        for token in tokens:
            if isinstance(token, str) and '/' in token:
                word, pos = token.rsplit('/', 1)  # '/' 기준으로 단어와 품사 분리
                if pos not in exclude_pos and word not in exclude_words:  # 품사와 단어가 제외 목록에 없으면 추가
                    filtered_tokens.append(token)
    except:
        continue  # 오류 발생 시 건너뛰기

# 빈도 계산
word_counts = Counter(filtered_tokens)

# 상위 100개 출력
print(word_counts.most_common(100))


[('하/VV', 729816), ('되/VV', 274910), ('금리/NNG', 217961), ('있/VV', 148501), ('상승/NNG', 114106), ('보/VV', 93570), ('오르/VV', 91863), ('시장/NNG', 86730), ('대출/NNG', 86058), ('경제/NNG', 81458), ('않/VX', 79352), ('하락/NNG', 74586), ('달러/NNG', 73648), ('전망/NNG', 71990), ('높/VA', 71801), ('인상/NNG', 63139), ('투자/NNG', 63122), ('경기/NNG', 60318), ('예상/NNG', 59783), ('위하/VV', 57045), ('따르/VV', 56781), ('기업/NNG', 56402), ('중/NNG', 55847), ('fed/NNG', 55159), ('가격/NNG', 53481), ('포인트/NNG', 51773), ('받/VV', 50423), ('수준/NNG', 48595), ('가능성/NNG', 48378), ('늘/VV', 47550), ('더/MAG', 47413), ('없/VA', 47322), ('대비/NNG', 45771), ('증가/NNG', 45276), ('증시/NNG', 44700), ('물가/NNG', 44654), ('은행/NNG', 44091), ('있/VA', 43738), ('지수/NNG', 41053), ('나오/VV', 39593), ('기록/NNG', 38786), ('기대/NNG', 38754), ('금융/NNG', 38720), ('우려/NNG', 38096), ('같/VA', 38007), ('발표/NNG', 37528), ('때/NNG', 37163), ('거래/NNG', 35441), ('줄/VV', 35321), ('인하/NNG', 34574), ('분석/NNG', 34273), ('나타나/VV', 34048), ('주가/NNG', 33997), ('규모/NNG', 3369

In [ ]:
# 전처리된 데이터를 새로운 CSV 파일로 저장
df.to_csv("filtered_real_data.csv", index=False)

# 저장된 파일 경로 출력
print("전처리된 데이터가 'filtered_processed_data.csv'로 저장되었습니다.")


전처리된 데이터가 'filtered_processed_data.csv'로 저장되었습니다.


In [ ]:
# 전체 데이터에서 토큰화된 단어 리스트를 모은 후 빈도 계산
#all_tokens = []
#for sublist in final_df['tokens']:
#    for token in sublist:
#        if isinstance(token, tuple):  # (단어, 품사) 형태라면
#            word, pos = token  # 튜플에서 단어와 품사를 분리
#            if word not in ["'", "N","/",","]:  # 작은따옴표 및 "N" 제거
#                all_tokens.append(word)  # 단어만 추가
#        else:  # 튜플이 아닌 경우
#            if token not in ["'", "N","/",","]:  # 작은따옴표 및 "N" 제거
#                all_tokens.append(token)  

#word_counts = Counter(all_tokens)

# 상위 100개 단어 출력
#print(word_counts.most_common(100))


In [28]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("real_data_test.csv")

# 'tagged' 컬럼의 샘플 데이터 출력
print(df['tagged'].head())  # 첫 5개의 샘플을 확인


0    [('국내', 'NNG'), ('채권전문가', 'NNG'), ('100', 'SN'...
1    [('사진', 'NNG'), ('AP', 'SL'), ('뉴욕증시', 'NNG'),...
2    [('미국', 'NNG'), ('뉴욕', 'NNG'), ('맨해튼', 'NNP'),...
3    [('정부', 'NNG'), ('는', 'JX'), ('소상공인', 'NNG'), ...
4    [('사진', 'NNG'), ('게티', 'NNP'), ('이미지', 'NNP'),...
Name: tagged, dtype: object


In [5]:
import pandas as pd

# 저장된 CSV 파일 불러오기
output_path = r"C:\Users\yoons\Desktop\금리예측프로젝트\filter\real_data_test.csv"
final_df = pd.read_csv(output_path)

# 'tagged' 컬럼의 첫 5개 값을 확인
print(final_df['tagged'].head())


0    [('국내', 'NNG'), ('채권전문가', 'NNG'), ('100', 'SN'...
1    [('사진', 'NNG'), ('AP', 'SL'), ('뉴욕증시', 'NNG'),...
2    [('미국', 'NNG'), ('뉴욕', 'NNG'), ('맨해튼', 'NNP'),...
3    [('정부', 'NNG'), ('는', 'JX'), ('소상공인', 'NNG'), ...
4    [('사진', 'NNG'), ('게티', 'NNP'), ('이미지', 'NNP'),...
Name: tagged, dtype: object
